In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
import csv
import random
from datasets import Dataset, DatasetDict, load_dataset

In [5]:
dataset = load_dataset("microsoft/orca-math-word-problems-200k")

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 200035
    })
})

In [7]:
updated_dataset = [{'Question': item['question'], 'Answer': item['answer']} for item in dataset['train']]

In [8]:
df=pd.DataFrame(updated_dataset)
file = df

que_list = []
ans_list = []

In [9]:
file.head()

,Question,Answer
0,Jungkook is the 5th place. Find the number of ...,"If Jungkook is in 5th place, then 4 people cro..."
1,A number divided by 10 is 6. Yoongi got the re...,"Let's call the certain number ""x"". According t..."
2,Dongju selects a piece of paper with a number ...,To find the second smallest and third smallest...
3,"You wanted to subtract 46 from a number, but y...",If you accidentally subtracted 59 instead of 4...
4,The length of one span of Jinseo is about 12 c...,If one span of Jinseo is about 12 centimeters ...


In [10]:
file.describe

<bound method NDFrame.describe of                                                  Question  \
0       Jungkook is the 5th place. Find the number of ...   
1       A number divided by 10 is 6. Yoongi got the re...   
2       Dongju selects a piece of paper with a number ...   
3       You wanted to subtract 46 from a number, but y...   
4       The length of one span of Jinseo is about 12 c...   
...                                                   ...   
200030  Adult tickets for a show cost 6.50 dollars, wh...   
200031  Your cell phone company offers three text-mess...   
200032  Steve invests in a circus production. The cost...   
200033  Your teacher is giving a test worth 200 points...   
200034  A plane flying with a tailwind at an altitude ...   

                                                   Answer  
0       If Jungkook is in 5th place, then 4 people cro...  
1       Let's call the certain number "x". According t...  
2       To find the second smallest and third smalles

In [11]:
for i in range(len(df)):    
    que_list.append(df['Question'][i])
    ans_list.append(df['Answer'][i])
    

In [12]:
# for line in file:
#         # skip first line
#         if line[0]=='Question':
#             continue
#         que_list.append(line[0])
#         ans_list.append(line[1])

In [13]:
que_list

['Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.',
 'A number divided by 10 is 6. Yoongi got the result by subtracting 15 from a certain number. What is the result he got?',
 'Dongju selects a piece of paper with a number written on it, and wants to make a three-digit number by placing the first selected number in the hundreds place, the second selected in the tens place, and the third selected in the units place. If the numbers written on each paper was 1, 6, and 8, respectively, find the sum of the second smallest and third smallest three-digit numbers that Dongju can make. However, you cannot select the same numbered paper multiple times.',
 'You wanted to subtract 46 from a number, but you accidentally subtract 59 and get 43. How much do you get from the correct calculation?',
 'The length of one span of Jinseo is about 12 centimeters (cm). When Jinseo measured the length of the shorter side of the bookshelf, it was about two 

In [14]:
intstructions_string = f""" Given is a Math word Problem, Understand the problem and give a very Logical Solution 
to the problem.
"""

In [15]:
example_template = lambda question, answer: f'''<s>[INST] {intstructions_string} \n{question} \n[/INST]\n''' + answer + "</s>"

In [16]:
example_list = []
for i in range(len(que_list)):
    example = example_template(que_list[i],ans_list[i])
    example_list.append(example)

In [17]:
print(example_list[10])

<s>[INST]  Given is a Math word Problem, Understand the problem and give a very Logical Solution 
to the problem.
 
This year, the age difference between Minsu and his mother is 28 years old, and after 13 years, the age of his mother will be twice that of Minsu. Find the age of Minsu this year. 
[/INST]
Let's denote Minsu's current age as M and his mother's current age as M_m.

According to the information given, we have two equations:

1) M_m - M = 28 (The age difference between Minsu and his mother is 28 years)

2) M_m + 13 = 2(M + 13) (After 13 years, the mother's age will be twice Minsu's age)

Now, let's solve these equations step by step.

From equation 1, we can express M_m in terms of M:

M_m = M + 28

Now, let's substitute M_m in equation 2 with the expression we found from equation 1:

(M + 28) + 13 = 2(M + 13)

Simplify the equation:

M + 41 = 2M + 26

Now, let's solve for M:

2M - M = 41 - 26
M = 15

So, Minsu is currently 15 years old.</s>


In [18]:
# create train/test split
test_index_list = random.sample(range(0, len(example_list)-1), 10000)

test_list = [example_list[index] for index in test_index_list]

for example in test_list:
    example_list.remove(example)

In [19]:
data = DatasetDict({'train':Dataset.from_dict({"example":example_list}), 'test':Dataset.from_dict({"example":test_list})})


In [20]:
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 190035
    })
    test: Dataset({
        features: ['example'],
        num_rows: 10000
    })
})

In [36]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

# Import Optimum libraries
import optimum  # Add this line to import the Optimum library

# Import Auto-Gptq libraries
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig 
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")


ImportError: Loading a GPTQ quantized model requires optimum (`pip install optimum`) and auto-gptq library (`pip install auto-gptq`)

In [1]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

NameError: name 'AutoTokenizer' is not defined

In [ ]:

model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "Great content, thank you!"
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

In [ ]:

model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

In [ ]:

# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["example"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)